<a href="https://colab.research.google.com/github/aaronszypulagastro/Marketing-Optimizer-Gaming-Industry-/blob/main/Marketing_Data_First_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import os, sys, random, warnings
from pathlib import Path

#Visualiserung
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='whitegrid') # globaler stil: weiß

# SCKIT LEARN: PIPELINE UND PREPROCESSING

  # Daten aufteilen in train/test, cross val berechnen
from sklearn.model_selection import train_test_split, cross_val_score
  # Nützlich für unterschiedliche preprocessing schritte in einem objekt
from sklearn.compose import ColumnTransformer
  # Kettet schritte (imputer , encoder/scaler, modell) zu einem sauberen workflow
from sklearn.pipeline import Pipeline
  # Dummy-Spalten, standartscaler für nomierte skala
from sklearn.preprocessing import OneHotEncoder, StandardScaler
  # Fehlwerte auffüllen
from sklearn.impute import SimpleImputer

# SCIKIT LEARN: MODELLE UND METRIKEN

  # Einfaches klassifikationsmodell
from sklearn.linear_model import LogisticRegression
  # Ensemble-bäume für Klassifikation und Regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
  # Metrics
from sklearn.metrics import (
    accuracy_score, f1_score, roc_auc_score, confusion_matrix, classification_report,
    mean_absolute_error, mean_squared_error, r2_score
)

# MODELLE SPEICHERN
import joblib

# Warnungen und Reproduzierbarkeit
  # Erster Seed (beliebige Zahl)
SEED = 42
  # Setzt Zufallsgeneratoren in Python, NumPy und TensorFlow
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
  # Unterdrückt Warnungen (enfernen beim Debugging :)
warnings.filterwarnings('ignore')

# PFADE
  # Arbeitsverzeichnis
PROJECT_DIR = Path.cwd()
  # Pfad zusammensetzen via '/'-Operator
DATA_DIR = PROJECT_DIR / 'data'
  # Zielordner für Grafiken/Modelle/Reports
RESULTS_DIR = PROJECT_DIR / 'results'
  # Legt results/ an falls nicht vorhanden
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
  # Kurzer Versionscheck
print(f'Using:\n pandas {pd.__version__}\n- sklearn ready\n- tensorflow {tf.__version__}')

Using:
 pandas 2.2.2
- sklearn ready
- tensorflow 2.19.0


In [3]:
# Einfügen der CSV_Datei
DATA_PATH = '/content/Social_Media_Advertising.csv'

# CSV einlesen
df = pd.read_csv(DATA_PATH)

# Form und erste Zeilen prüfen
print('Shape:', df.shape) # Zeile, Spalten
print(df.head()) # Erste Zeilen
print(df.dtypes) # Datentype pro Spalte

Shape: (300000, 16)
   Campaign_ID Target_Audience  ...        Date         Company
0       529013       Men 35-44  ...  2022-02-25      Aura Align
1       275352     Women 45-60  ...  2022-05-12  Hearth Harmony
2       692322       Men 45-60  ...  2022-06-19   Cyber Circuit
3       675757       Men 25-34  ...  2022-09-08       Well Wish
4       535900       Men 45-60  ...  2022-08-24  Hearth Harmony

[5 rows x 16 columns]
Campaign_ID           int64
Target_Audience      object
Campaign_Goal        object
Duration             object
Channel_Used         object
Conversion_Rate     float64
Acquisition_Cost     object
ROI                 float64
Location             object
Language             object
Clicks                int64
Impressions           int64
Engagement_Score      int64
Customer_Segment     object
Date                 object
Company              object
dtype: object


In [6]:
# Zielvariable definieren

# Schwelle definieren (hier 5% Conversion-Rate als Beispiel)
threshold = 0.05

df['Success'] = (df['Conversion_Rate'] >= threshold).astype(int)

print(df['Success'].value_counts(normalize=True)) # Wie viel Prozent der Kampagne war erfolgreich?
print(df[['Conversion_Rate', 'Success']].head(10))

Success
1    0.750603
0    0.249397
Name: proportion, dtype: float64
   Conversion_Rate  Success
0             0.15        1
1             0.01        0
2             0.08        1
3             0.03        0
4             0.13        1
5             0.02        0
6             0.10        1
7             0.10        1
8             0.14        1
9             0.04        0


In [7]:
# Features (X) und Ziel (y)

# Spalten die wir NICHT als Input nutzen (IDs, Zielspalten)

  # Modell 'rät' die schwelle wenn Conversion_Rate drin belibt
drop_cols = ['Campaign_ID', 'Company', 'Date', 'Conversion_Rate', 'Success' ]

# Features (X) und Zielvariable (y) definieren
X = df.drop(drop_cols, axis=1)
y = df['Success']

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (300000, 13)
y shape: (300000,)


In [15]:
# Train-Test Split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_honest, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print('Train size:', X_train.shape, 'Test size:', X_test.shape)


Train size: (240000, 7) Test size: (60000, 7)


In [18]:
# Preprocessing-Pipeline

# 1) Spalten nach Typ trennen
cat_cols = X_honest.columns.tolist()

print('Kategorisch:', cat_cols)


# 2) Pipeline für kategorische Spalten
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # fehlende Werte durch Modalwert ersetzen
    ('onehot', OneHotEncoder(handle_unknown='ignore')) # One-Hot-Encoding (z.B. 'Instagram'zu 0/1 Spalten )
])

# 3) Alles zusammenbauen
preprocessor = ColumnTransformer(
    transformers=
    [
    ('cat', categorical_transformer, cat_cols)
    ]
)

Kategorisch: ['Target_Audience', 'Campaign_Goal', 'Duration', 'Channel_Used', 'Location', 'Language', 'Customer_Segment']


In [19]:
# Logistic Regression als erstes Baseline-Modell

# Pipeline + Modell kombinieren
clf = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression(max_iter=1000))
])

# Modell trainieren
clf.fit(X_train, y_train)

# Vorhersagen
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:, 1] # für ROC-AUC

In [20]:
# Evaluation

# Accuracy
print('Accuracy', accuracy_score(y_test, y_pred))

# F1-Score (besser bei unbalancierten Klassen)
print('F1-Score', f1_score(y_test, y_pred))

# ROC-AUC
print('ROC-AUC', roc_auc_score(y_test, y_proba))

# Confusion Matrix
print('Confusion Matrix\n', confusion_matrix(y_test, y_pred))

# Classification Report (Precision, Recall, F1 pro Klasse)
print('\nClassification Report:\n', classification_report(y_test, y_pred))


Accuracy 0.7506
F1-Score 0.8575345595795727
ROC-AUC 0.5022733276148987
Confusion Matrix
 [[    0 14964]
 [    0 45036]]

Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00     14964
           1       0.75      1.00      0.86     45036

    accuracy                           0.75     60000
   macro avg       0.38      0.50      0.43     60000
weighted avg       0.56      0.75      0.64     60000



**Fazit1.1: Vermutlicher Data Leakage, an was liegt das? Im ersten Versuch mit allen Features erzielte das Modell eine perfekte Accuracy von 1.0. Das war jedoch ein Hinweis auf Data Leakage, da Variablen wie ROI oder Acquisition_Cost direkt mit der Zielgröße Conversion_Rate zusammenhängen. Damit war das Modell zwar mathematisch „perfekt“, aber praktisch nicht realistisch einsetzbar**

**Fazit1.2: Im zweiten Versuch wurden nur „ehrliche“ Features genutzt, die vor Kampagnenstart bekannt sind (z. B. Zielgruppe, Kanal, Sprache, Segment). Dadurch sank die Modell-Performance auf ca. 75 % Accuracy, aber dieses Ergebnis ist realistisch und praxisrelevant. Es zeigt, dass die Auswahl von Zielgruppe, Kanal oder Kampagnenziel einen messbaren Einfluss auf den Erfolg hat – auch wenn das Modell die „nicht erfolgreichen“ Kampagnen aktuell noch schlecht unterscheidet.**

In [12]:
# Wir nehmen nur ehrliche Features (keine 'Outcome'-Variablen)
honest_features = [
    'Target_Audience', 'Campaign_Goal', 'Duration',
    'Channel_Used', 'Location', 'Language', 'Customer_Segment'
]

X_honest = df[honest_features]
y = df['Success']

print('Neue Feature-Shape:', X_honest.shape)
print('y distribution:', y.value_counts(normalize=True))

Neue Feature-Shape: (300000, 7)
y distribution: Success
1    0.750603
0    0.249397
Name: proportion, dtype: float64


In [ ]:
# Nächste Schritte:
# Training meherer Modelle auf denselben Preprocessing Schritten und denselben Daten
# Ziel: Faire, vergleichbare Metriken
# Guter Vergleich welches Modell mit den Features performt

from xgboost import XGBClassifier #Gradient Boosting
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report

models = {
    'LogReg {balanced}': LogisticRegression(class_weight='balanced', max_iter=1000),
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'XGBoost (balanced)': XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        scale_pos_weight=0.33,  # Ungleichgewicht ausgleichen
        use_label_encoder=False,
        eval_metric="logloss",
        random_state=42
    )
}

results = {}

for name, model in models.items():
  clf = Pipeline(
      steps=[
          ('preprocessor', preprocessor),
          ('model', model)
  ])

  # Training
  clf.fit(X_train, y_train)

  # Vorhersagen
  y_pred = clf.predict(X_test)
  y_proba = clf.predict_proba(X_test)[:, 1]

  # Metriken speichern
  results[name] = {
      'Accuracy': accuracy_score(y_test, y_pred),
      'F1-Score': f1_score(y_test, y_pred),
      'ROC-AUC': roc_auc_score(y_test, y_proba)
  }

  print(f'\n==== {name} ====')
  print('Accuracy:', results[name]['Accuracy'])
  print('F1-Score:', results[name]['F1-Score'])
  print('ROC-AUC:', results[name]['ROC-AUC'])
  print('\nClassification Report:\n', classification_report(y_test, y_pred))

# Übersichtliche Tabelle
results_df = pd.DataFrame(results).T
print('\n=== Modellvergleich ===')
print(results_df)



==== LogReg {balanced} ====
Accuracy: 0.5101166666666667
F1-Score: 0.6140878356200354
ROC-AUC: 0.5022398918312261

Classification Report:
               precision    recall  f1-score   support

           0       0.25      0.48      0.33     14964
           1       0.75      0.52      0.61     45036

    accuracy                           0.51     60000
   macro avg       0.50      0.50      0.47     60000
weighted avg       0.63      0.51      0.54     60000



FAZIT:
Logistic Regression: zu simpel → liefert Zufallsergebnisse.

Random Forest: liefert aktuell die sinnvollsten Ergebnisse → erkennt Muster, beide Klassen werden besser behandelt.

XGBoost: wirkt super, aber nur, weil es die Mehrheitsklasse bevorzugt. Ohne Anpassung (z. B. scale_pos_weight) ist es nicht vertrauenswürdig.**



** Nächster Schritt:
Einbau scale_pos_weight
und Bäume verringern da nicht genug Rechenleistung (dauert zu lange :( )**